# Implementierung eines Experten­systems für Tiere

Ein Beispiel aus dem [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

In diesem Beispiel werden wir ein einfaches wissensbasiertes System implementieren, um ein Tier anhand einiger körperlicher Merkmale zu bestimmen. Das System kann durch den folgenden AND-OR-Baum dargestellt werden (dies ist ein Teil des gesamten Baums, wir können problemlos weitere Regeln hinzufügen):

![](../../../../../../translated_images/de/AND-OR-Tree.5592d2c70187f283.webp)


## Unser eigenes Expertensystem-Shell mit Rückwärtsverkettung

Lassen Sie uns versuchen, eine einfache Sprache für die Wissensrepräsentation basierend auf Produktionsregeln zu definieren. Wir verwenden Python-Klassen als Schlüsselwörter, um Regeln zu definieren. Es gibt im Wesentlichen 3 Arten von Klassen:
* `Ask` repräsentiert eine Frage, die dem Benutzer gestellt werden muss. Es enthält die Menge möglicher Antworten.
* `If` repräsentiert eine Regel und ist nur ein syntaktischer Zucker, um den Inhalt der Regel zu speichern.
* `AND`/`OR` sind Klassen, um AND/OR-Verzweigungen des Baums darzustellen. Sie speichern nur die Liste der Argumente darin. Zur Vereinfachung des Codes ist die gesamte Funktionalität in der Oberklasse `Content` definiert.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

In unserem System würde das Arbeitsgedächtnis die Liste der **Fakten** als **Attribut-Wert-Paare** enthalten. Die Wissensbasis kann als ein großes Wörterbuch definiert werden, das Aktionen (neue Fakten, die in das Arbeitsgedächtnis eingefügt werden sollen) auf Bedingungen abbildet, die als UND-ODER-Ausdrücke dargestellt sind. Außerdem können einige Fakten `Ask`-ed werden.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Um die Rückwärtsinferenz durchzuführen, definieren wir die Klasse `Knowledgebase`. Sie wird enthalten:
* Arbeitsspeicher `memory` – ein Wörterbuch, das Attribute auf Werte abbildet
* Knowledgebase-`rules` im oben definierten Format

Zwei Hauptmethoden sind:
* `get`, um den Wert eines Attributs zu erhalten, wobei bei Bedarf eine Inferenz durchgeführt wird. Zum Beispiel würde `get('color')` den Wert eines Farb-Slots abfragen (es wird bei Bedarf gefragt und der Wert für die spätere Verwendung im Arbeitsspeicher gespeichert). Wenn wir `get('color:blue')` abfragen, wird nach einer Farbe gefragt und dann abhängig von der Farbe ein `y`/`n`-Wert zurückgegeben.
* `eval` führt die eigentliche Inferenz durch, d.h. durchläuft den AND/OR-Baum, bewertet Teilziele usw.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Definieren wir nun unsere Tierwissensdatenbank und führen die Beratung durch. Beachten Sie, dass Ihnen bei diesem Aufruf Fragen gestellt werden. Sie können mit `y`/`n` für Ja-Nein-Fragen antworten oder eine Zahl (0..N) für Fragen mit längeren Mehrfachauswahlantworten angeben.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Verwendung von Experta für Vorwärts-Schlussfolgerungen

Im nächsten Beispiel werden wir versuchen, Vorwärts-Schlussfolgerungen mit einer der Bibliotheken für Wissensrepräsentation, [Experta](https://github.com/nilp0inter/experta), zu implementieren. **Experta** ist eine Bibliothek zur Erstellung von Vorwärts-Schlusssystemen in Python, die so gestaltet ist, dass sie dem klassischen alten System [CLIPS](http://www.clipsrules.net/index.html) ähnelt.

Wir hätten die Vorwärtsverkettung auch selbst ohne große Probleme implementieren können, aber naive Implementierungen sind normalerweise nicht sehr effizient. Für ein effektiveres Regelmatching wird ein spezieller Algorithmus namens [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) verwendet.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Wir definieren unser System als eine Klasse, die von `KnowledgeEngine` erbt. Jede Regel wird durch eine separate Funktion mit der `@Rule`-Annotation definiert, die angibt, wann die Regel ausgelöst werden soll. Innerhalb der Regel können wir mit der Funktion `declare` neue Fakten hinzufügen, und das Hinzufügen dieser Fakten führt dazu, dass weitere Regeln durch die vorwärtsgerichtete Inferenzmaschine aufgerufen werden.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Sobald wir eine Wissensbasis definiert haben, füllen wir unser Arbeitsgedächtnis mit einigen Anfangsfakten und rufen dann die `run()`-Methode auf, um die Inferenz durchzuführen. Sie können als Ergebnis sehen, dass neue abgeleitete Fakten dem Arbeitsgedächtnis hinzugefügt werden, einschließlich der endgültigen Tatsache über das Tier (wenn wir alle anfänglichen Fakten korrekt eingerichtet haben).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ursprungssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
